In [1]:
import json
import time

from tqdm.auto import tqdm
import requests
import pandas as pd
import geopandas as gpd
from docx import Document


tqdm.pandas()
client_cert = "20250129_IDU_ADGX_LeonTur.crt"
ca_cert = "onti-ca.crt"
client_key = "DECFILE"

In [4]:
def construct_prompt(context):
    """
    Формирует промпт для модели на основе контекста.
    """

    context_str = "\n".join(context) if isinstance(context, list) else str(context)
    prompt = f"""
        Найди названия в тексте {context_str}.
        Названия должны принадлежать населеённым пунктам или административным единицам, которые физически представлены на территории.
        Приведи названия в начальную форму. Добавь как можно больше названий на основе контекста. В качестве ответа напиши только название. Если название не представлено в тексте или текст отсутствует или не был предоставлен, в качестве ответа дай ''
        Если названий больше одного, сохрани названия элементами в списке через запятую [Название1, Название2, Название3]
        """
    return prompt

In [5]:
def request_from_llm(prompt: str) -> str:
    """
    Function extracts prompt from llm api
    Args:
        prompt (str): prompt to execute
    Returns:
        str: request from llm
    """

    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama3.3",
        "prompt": prompt,
        "stream": False,
    }
    response = requests.post(
        url="http://a.dgx:11434/api/generate",
        headers=headers,
        data=json.dumps(data),
        cert=(client_cert, client_key),
        verify=ca_cert
    )
    if response.status_code == 200:
        return response.json()["response"]
    return ""

In [6]:
def extract_ner(text: str) -> str:

    prompt = construct_prompt(text)
    response_text = request_from_llm(prompt)
    return response_text
test = extract_ner("В Санкт-Петерубрге сегодня несолнечно, холодно, депрессивно и вообще повеситься хочется")
test

'Санкт-Петербург'

In [7]:
test_doc = Document("стратегия_2035.docx")

In [8]:
list_for_save = []
current_par = []
switcher = False
privios_par = None
total_parapgraphs = []
for paragraph in tqdm(test_doc.paragraphs, desc="Processing texts"):
    if paragraph.style.name == "ConsPlusTitle":
        if switcher:
            current_par = [paragraph.text]
            total_parapgraphs.append(paragraph.text)
            switcher = False
        else:
            current_par.append(paragraph.text)
            total_parapgraphs.append(paragraph.text)
    else:
        if not switcher:
            switcher = True
        if paragraph.text:
            list_for_save.append(
                {
                    "text": paragraph.text,
                    "title": "".join(current_par),
                    "NER": extract_ner(paragraph.text),
                }
            )

Processing texts:   0%|          | 0/1493 [00:00<?, ?it/s]

In [9]:
df = pd.DataFrame.from_records(list_for_save)

In [11]:
df.to_csv("with_names.csv")

In [32]:
name = " ".join(df[df["title"].isin([""])]["text"].to_list())

In [33]:
name

'ПРИЛОЖЕНИЕ к Закону Санкт-Петербурга "О Стратегии социально-экономического развития Санкт-Петербурга на период до 2035 года" от 19.12.2018 N 771-164 '

In [33]:
def geocode_all(
        ner: str
) -> list | None:
    if pd.isna(ner) or not ner:
        return None
    ner_list = ner.split(", ")
    res_list = []
    for i in ner_list:
        time.sleep(1)
        res_list.append(gpd.tools.geocode(i, provider="arcgis"))
    return res_list

In [27]:
df.loc[df["NER"].str.contains("\n"), "NER"] = None
df["NER"].replace("", None, inplace=True)

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [28]:
df["NER"]

0                                                 None
1                                                 None
2                                                   ''
3       Поскольку текст не был предоставлен, ответ: ''
4                                                   ''
                             ...                      
1260                                   Санкт-Петербург
1261                                   Санкт-Петербург
1262                                   Санкт-Петербург
1263             Санкт-Петербург, Российская Федерация
1264                                   Санкт-Петербург
Name: NER, Length: 1265, dtype: object

In [34]:
df["addresses"] = df["NER"].progress_apply(geocode_all)

  0%|          | 0/1265 [00:00<?, ?it/s]

In [36]:
df

,text,title,NER,addresses
0,ПРИЛОЖЕНИЕ,,None,None
1,к Закону Санкт-Петербурга,,None,None
2,"""О Стратегии",,'',[[geometry]]
3,социально-экономического,,"Поскольку текст не был предоставлен, ответ: ''","[[geometry], [geometry]]"
4,развития Санкт-Петербурга,,'',[[geometry]]
...,...,...,...,...
1260,улучшение инвестиционного климата Санкт-Петерб...,"10. Оценка финансовых ресурсов, необходимых дл...",Санкт-Петербург,[[geometry]]
1261,Исходя из оценки ретроспективных данных о взаи...,"10. Оценка финансовых ресурсов, необходимых дл...",Санкт-Петербург,[[geometry]]
1262,Реализация Стратегии 2035 осуществляется посре...,11. Информация о государственных программахСан...,Санкт-Петербург,[[geometry]]
1263,Государственные программы Санкт-Петербурга раз...,11. Информация о государственных программахСан...,"Санкт-Петербург, Российская Федерация","[[geometry], [geometry]]"


In [60]:
from shapely import to_wkt
from shapely.geometry import MultiPoint

def dump_to_wkt(geometry_list: list) -> MultiPoint | None:
    try:
        if geometry_list:
            res = [i["geometry"].iloc[0] for i in geometry_list if i["address"].iloc[0]]
            if len(res) > 1:
                return MultiPoint(points=res)
            elif len(res) == 1:
                return res[0]
            return None
    except Exception as e:
        print(e)
        print(geometry_list)
        print(res)
df["geometry"] = df["addresses"].apply(dump_to_wkt)

In [63]:
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs=4326)
gdf

,text,title,NER,addresses,geometry
0,ПРИЛОЖЕНИЕ,,None,None,None
1,к Закону Санкт-Петербурга,,None,None,None
2,"""О Стратегии",,'',[[geometry]],None
3,социально-экономического,,"Поскольку текст не был предоставлен, ответ: ''","[[geometry], [geometry]]",POINT (129.71138 62.0176)
4,развития Санкт-Петербурга,,'',[[geometry]],None
...,...,...,...,...,...
1260,улучшение инвестиционного климата Санкт-Петерб...,"10. Оценка финансовых ресурсов, необходимых дл...",Санкт-Петербург,[[geometry]],POINT (30.31248 59.93848)
1261,Исходя из оценки ретроспективных данных о взаи...,"10. Оценка финансовых ресурсов, необходимых дл...",Санкт-Петербург,[[geometry]],POINT (30.31248 59.93848)
1262,Реализация Стратегии 2035 осуществляется посре...,11. Информация о государственных программахСан...,Санкт-Петербург,[[geometry]],POINT (30.31248 59.93848)
1263,Государственные программы Санкт-Петербурга раз...,11. Информация о государственных программахСан...,"Санкт-Петербург, Российская Федерация","[[geometry], [geometry]]","MULTIPOINT ((30.31248 59.93848), (96.80537 61...."
